<a href="https://colab.research.google.com/github/mayaelabed/ETL/blob/main/Copie_de_Atelier_3_ETL_avanc%C3%A9_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ETL avancé

## Extraire à partir du web (web scrapping)

Le web est une source importante de données publiques. L'extraction des données à partir des pages HTML est appelé web scraping. Il existe plusieurs librairies dans python pour effectuer le web scraping. Dans cet atelier, nous allons utiliser **beautifulsoup4** avec **html5lib** comme parser html5.La bibliothèque requests sert à charger le contenu d'un site web dans un objet response. 

In [2]:
# dans colab ces bibliothèques sont préinstallées
#!pip install html5lib
#!pip install requests
#!pip install beautifulsoup4

In [3]:
from bs4 import BeautifulSoup
import requests

On commence par charger le contenu d'une page web en utilisant **requests**

In [4]:
url = "https://fr.wikipedia.org/wiki/Web_scraping"
html = requests.get(url).text
html

'<!DOCTYPE html>\n<html class="client-nojs" lang="fr" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>Web scraping — Wikipédia</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":[",\\t.","\xa0\\t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","janvier","février","mars","avril","mai","juin","juillet","août","septembre","octobre","novembre","décembre"],"wgRequestId":"bc5d3b21-fd08-4f07-927d-d0669aed7ecc","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Web_scraping","wgTitle":"Web scraping","wgCurRevisionId":175102277,"wgRevisionId":175102277,"wgArticleId":2778658,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Article manquant de références depuis avril 2020","Article manquant de références/Liste complète","Article contenant un appel à traduction en anglais","Po

On instancie un objet de la classe **BeautifulSoup** avec html5lib comme parser.

In [5]:
soup = BeautifulSoup(html, 'html5lib')

Une fois l'objet est créé, on peut extraire le contenu souhaité en utilisant les méthodes de la bibliothèque. https://www.crummy.com/software/BeautifulSoup/bs4/doc/

Exemple 1 : Extraire la première paragraphe.
On utilise la fonction `find` pour extraire la première occurence d'une balise donnée.



In [6]:
paragraph = soup.find("p").getText()
paragraph


"Si vous disposez d'ouvrages ou d'articles de référence ou si vous connaissez des sites web de qualité traitant du thème abordé ici, merci de compléter l'article en donnant les références utiles à sa vérifiabilité et en les liant à la section «\xa0Notes et références\xa0» \n"

Exemple 2 : Extraire toutes les paragraphes. 
`find_all` permet d'extraire toutes les occurrences d'une balise dans la page html.Pour cela, on utilise la technique de *list comprehension*. 



In [7]:
allparagraphs=[p.getText() for p in soup.find_all("p")]
allparagraphs

["Si vous disposez d'ouvrages ou d'articles de référence ou si vous connaissez des sites web de qualité traitant du thème abordé ici, merci de compléter l'article en donnant les références utiles à sa vérifiabilité et en les liant à la section «\xa0Notes et références\xa0» \n",
 'En pratique\xa0: Quelles sources sont attendues\xa0? Comment ajouter mes sources\xa0?\n',
 "Le web scraping (parfois appelé harvesting) est une technique d'extraction du contenu de sites Web, via un script ou un programme, dans le but de le transformer pour permettre son utilisation dans un autre contexte, par exemple le référencement[1].\n",
 "Cela permet de récupérer le contenu d’une page web en vue d'en réutiliser le contenu. Cette opération se pratique le plus souvent de façon automatique, qui permet de constituer des pages à bon compte. Cette pratique n'a pas très bonne presse chez les contributeurs authentiques car elle peut être assimilée à un pillage.\n",
 "Google Actualités, en agrégeant sans autorisa

## Charger dans un Base de données

Les bases de données sont les moyens les plus souvent utilisées pour la sauvegarde des données. Dans cette partie, nous allons voir comment charger des données dans un BD **sqlite**.

On commence par créer une connexion au SGBD.

In [8]:
import sqlite3

In [9]:
conn = sqlite3.connect('session.db')

Ensuite, on peut créer une table dans bd.

In [10]:
try:
    conn.execute('''
         CREATE TABLE wiki
         (ID         INTEGER PRIMARY KEY AUTOINCREMENT,
         title        TEXT    NOT NULL,
         content        TEXT    NOT NULL);''')
    print ("Table created successfully");
except Exception as e:
    print(str(e))
    print('Table Creation Failed!!!!!')

Table created successfully


Une fois la table est crée, on peut insérer des lignes dedans.

In [11]:
title= "Python"
content="Python est un langage de programmation haut niveau...."
conn.execute('INSERT INTO wiki (title, content) VALUES (?, ?)',(title,content))

On peut consulter les lignes d'une table.

In [12]:
rows = conn.cursor().execute('Select * from wiki')

for row in rows:
    print(row)

(1, 'Python', 'Python est un langage de programmation haut niveau....')


# Travail à faire

Utiliser la technique *web scrapping * pour charger le maximum de définitions à partir de wikipédia dans une BD sqlite. Voici l'algorithme de L'ETL :


1.   Extraire le titre avec la balise h1
2.   Extraire la définition on utilisant le texte de la paragraphe de la `div` dont l'attribut `id` est `'mw-content-text'`

3. Enregistrer une nouvelle ligne dans la BD
4. Extraire les liens hypertextes dans la `div` dont l'attribut `id` est `'bodyContent'`

5. choisir un lien hypertexte de la forme `href=re.compile('^(/wiki/)((?!:).)*$')` parmis les liens obtenus
6. refaire les étapes précédantes en utilisant le nuveau lien.



In [21]:
t= soup.find("h1").getText()
t

'Web scraping'

In [20]:
x=soup.find('div').find(id ='mw-content-text')
x

<div class="mw-content-ltr" dir="ltr" id="mw-content-text" lang="fr"><div class="mw-parser-output"><div class="bandeau-container bandeau-article bandeau-niveau-modere plainlinks bandeau-container metadata ambox"><div class="floatright"><a href="/wiki/Aide:Bandeau" title="Si ce bandeau n'est plus pertinent, retirez-le. Cliquez ici pour en savoir plus sur les bandeaux."><img alt="Si ce bandeau n'est plus pertinent, retirez-le. Cliquez ici pour en savoir plus sur les bandeaux." class="noviewer" data-file-height="512" data-file-width="512" decoding="async" height="12" src="//upload.wikimedia.org/wikipedia/commons/thumb/3/38/Info_Simple.svg/12px-Info_Simple.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/3/38/Info_Simple.svg/18px-Info_Simple.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/3/38/Info_Simple.svg/24px-Info_Simple.svg.png 2x" width="12"/></a></div><div class="bandeau-cell bandeau-icone" style="display:table-cell;padding-right:0.5em"><a class="image" 

In [17]:
title= "jee"
content="jee est un langage de programmation haut niveau...."
conn.execute('INSERT INTO wiki (title, content) VALUES (?, ?)',(title,content))

In [18]:
rows = conn.cursor().execute('Select * from wiki')

for row in rows:
    print(row)

(1, 'Python', 'Python est un langage de programmation haut niveau....')
(2, 'java', 'java est un langage de programmation haut niveau....')
(3, 'jee', 'jee est un langage de programmation haut niveau....')


In [19]:
h =soup.find('div').find(id ='bodyContent')
h 

<div class="mw-body-content" id="bodyContent">
		<div class="noprint" id="siteSub">Un article de Wikipédia, l'encyclopédie libre.</div>
		<div id="contentSub"></div>
		<div id="contentSub2"></div>
		
		<div class="mw-content-ltr" dir="ltr" id="mw-content-text" lang="fr"><div class="mw-parser-output"><div class="bandeau-container bandeau-article bandeau-niveau-modere plainlinks bandeau-container metadata ambox"><div class="floatright"><a href="/wiki/Aide:Bandeau" title="Si ce bandeau n'est plus pertinent, retirez-le. Cliquez ici pour en savoir plus sur les bandeaux."><img alt="Si ce bandeau n'est plus pertinent, retirez-le. Cliquez ici pour en savoir plus sur les bandeaux." class="noviewer" data-file-height="512" data-file-width="512" decoding="async" height="12" src="//upload.wikimedia.org/wikipedia/commons/thumb/3/38/Info_Simple.svg/12px-Info_Simple.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/3/38/Info_Simple.svg/18px-Info_Simple.svg.png 1.5x, //upload.wikimedia.or

In [24]:
import re

In [27]:
href=h.find_all('a', href=re.compile('^(/wiki/)((?!:).)*$'))
href

[<a class="mw-redirect" href="/wiki/Site_Web" title="Site Web">sites Web</a>,
 <a href="/wiki/Langage_de_script" title="Langage de script">script</a>,
 <a href="/wiki/Programme_informatique" title="Programme informatique">programme</a>,
 <a href="/wiki/R%C3%A9f%C3%A9rencement" title="Référencement">référencement</a>,
 <a href="/wiki/Google_Actualit%C3%A9s" title="Google Actualités">Google Actualités</a>,
 <a href="/wiki/Int%C3%A9gration_d%27applications_d%27entreprise" title="Intégration d'applications d'entreprise">intégrations d'applications d'entreprise</a>,
 <a href="/wiki/LinkedIn" title="LinkedIn">LinkedIn</a>,
 <a href="/wiki/Cour_d%27appel_des_%C3%89tats-Unis_pour_le_neuvi%C3%A8me_circuit" title="Cour d'appel des États-Unis pour le neuvième circuit">Cour d'appel des États-Unis pour le neuvième circuit</a>,
 <a href="/wiki/Cour_supr%C3%AAme_des_%C3%89tats-Unis" title="Cour suprême des États-Unis">Cour suprême</a>,
 <a href="/wiki/Cour_de_justice_de_l%27Union_europ%C3%A9enne" tit

In [28]:
from bs4 import BeautifulSoup
import requests

In [30]:
url = "https://www.facebook.com/maya.elabed.31/"
html = requests.get(url).text
html

'<!DOCTYPE html>\n<html lang="en" id="facebook" class="no_js">\n<head><meta charset="utf-8" /><meta name="referrer" content="default" id="meta_referrer" /><script nonce="XxA8oFgA">window._cstart=+new Date();</script><script nonce="XxA8oFgA">function envFlush(a){function b(b){for(var c in a)b[c]=a[c]}window.requireLazy?window.requireLazy(["Env"],b):(window.Env=window.Env||{},b(window.Env))}envFlush({"ajaxpipe_token":"AXi2q1p3Kf88gyuEIHc","timeslice_heartbeat_config":{"pollIntervalMs":33,"idleGapThresholdMs":60,"ignoredTimesliceNames":{"requestAnimationFrame":true,"Event listenHandler mousemove":true,"Event listenHandler mouseover":true,"Event listenHandler mouseout":true,"Event listenHandler scroll":true},"isHeartbeatEnabled":true,"isArtilleryOn":false},"shouldLogCounters":true,"timeslice_categories":{"react_render":true,"reflow":true},"sample_continuation_stacktraces":true,"dom_mutation_flag":true,"stack_trace_limit":30,"timesliceBufferSize":5000,"show_invariant_decoder":false,"compat_

In [33]:
soup = BeautifulSoup(html, 'html5lib')

In [38]:
paragraph = soup.find("h1").getText()
paragraph

'Facebook'

In [50]:
fb=soup.find('div').find(id ='mount_0_0')
fb

In [61]:
import sqlite3

In [65]:
conn = sqlite3.connect('session.db')
conn= sqlite3.connect('cache.db', timeout=10)


In [66]:
try:
    conn.execute('''
         CREATE TABLE facebook
         (ID         INTEGER PRIMARY KEY AUTOINCREMENT,
         title        TEXT    NOT NULL,
         content        TEXT    NOT NULL);''')
    print ("Table created successfully");
except Exception as e:
    print(str(e))
    print('Table Creation Failed!!!!!')

Table created successfully


In [71]:
title= "fb"
content="facebook est créer par mark zuckerberg...."
conn.execute('INSERT INTO facebook (title, content) VALUES (?, ?)',(title,content))

In [72]:
rows = conn.cursor().execute('Select * from facebook')

for row in rows:
    print(row)

(1, 'fb', 'fb est un langage de programmation haut niveau....')
(2, 'fb', 'fb est créer par mark zuckerberg....')
(3, 'fb', 'facebook est créer par mark zuckerberg....')
